# Model Training
### Load processed data, train models, and evaluate models

In [6]:
!pip install tf

In [2]:
# Data = {X_train:, y_train:, y_test:, X_test:}

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
def model_train_eval(model, data, parameters):    
    model_temp=GridSearchCV(model, parameters, scoring='accuracy').fit(data["X_train"], data["y_train"])
    y_pred=model_temp.predict(data["X_test"])
    mse=mean_squared_error(data["y_test"], y_pred)
    print("Mean Squared Error: ", mse)

In [9]:
from sklearn.linear_model import LinearRegression
# The data here needs to be imported from the other data collection notebook
model_train_eval(LinearRegression(), data, {})

NameError: name 'data' is not defined

In [4]:
from sklearn.neural_network import MLPRegressor

model_train_eval(MLPRegressor(), data, {})

NameError: name 'data' is not defined

In [8]:
import tf
